In [88]:
"""You can define global variables here """

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib
import pandas as pd

global model # We want to use the same model throughout the interface
global company_num

def start( company, name ):
    """Called once at the beginning of testing period.
        Company is the number of the company you are predicting (0-2), or 3 if you are to
        predict the value of all companies.
        Name is the (path and) name of your model.
        You should load the model you use throughout here (no training to take place) """
    print("Starting model of company %d" % company)
    file_path = "models/" + name
    print("Loading model from \"" + file_path + "\"")
    from keras.models import load_model
    global model 
    global company_num
    company_num = company
    model = load_model(file_path)
    return


def predict( stock_prices, market_analysis, market_segments, info ):
    """Called on consecutive days of the test period (when stock markets are open).
       The arguments are provided as lists of tuples, ordered in an increasing
       order by date (i.e., the last tuple in stock_prices and info, are for the day you are
       predicting. You are not allowed to change the arguments.

       You are to predict the stock-price movement for the date indicated by the last record in
       the info list. Note, that the lists also contain information about older dates, but you are
       free to ignore them if not using them.

       Returns a 3-list of predictions (company 0, 1, 2, respectively). Each value is True if
       prediction is that stock price for respective company will go up, otherwise False
       (i.e. if you think company 0 will go up, but not 1 and 2, then return [True, False, False]).
       If you are only predicting a single company, set the respective field but keep the others False
       (i.e. if company is number 1 and you predict its stock will go up, then return [False, True, False]).
    """
    prediction = [False, False, False]
    
    
    # If it is a single company we run this...
    if company_num != 3:
        # Getting the info of today
        info_today = info[len(info) - 1]
        print(info_today)
    
        # Getting stock_price of today
        stock_today = stock_prices[len(stock_prices) - 1]
        stock_yesterday = stock_prices[len(stock_prices) - 2]
        
        print(stock_today)
        print(stock_yesterday)
        
        # Get the info we use in our model
        X = info_today[4:]
        
        if company_num != 1:
            # IT = [1,0]
            X = np.hstack((X, [1, 0]))
        else:
            X = np.hstack((X, [0, 1]))
        
        # Make the stock price binary based on yesterday
        if stock_today[4] > stock_yesterday[4]:
            X = np.hstack((X, [1]))
        else:
            X = np.hstack((X, [0]))
        
        scaler = joblib.load("models/scaler Company " + str(company_num) + ".save")
        X = X.reshape(1, -1)
        X_test = scaler.transform(X)
        X_test = np.array(X_test[0][:-1])
        print(X)
        print(X_test)
        
        predict = model.predict_classes(X_test.reshape(1,-1))
        print(predict[0][0])
        if predict[0][0] == 1:
            prediction[company_num] = True
    return prediction

def end():
    """Called once at the end of the testing period. Optional if you do anything here."""
    ...
    return


def main():
    
    info_data = pd.read_csv("data/info.txt", sep='\s+')
    market_analysis = pd.read_csv("data/market_analysis.txt", sep='\s+')
    market_segments = pd.read_csv("data/market_segments.txt", sep='\s+')
    stock_prices = pd.read_csv("data/stock_prices.txt", sep='\s+')
    
    days_back = 1
    
    info_0 = info_data[info_data['company'] == 0]
    info_0 = info_0[-days_back:]
    info_0 = [tuple(x) for x in info_0.values]
    stock_prices_0 = stock_prices[stock_prices['company'] == 0]
    stock_prices_0 = stock_prices_0[-days_back:]
    stock_prices_0 = [tuple(x) for x in stock_prices_0.values]
    
    start(0, "c1.h5")
    for i in range(days_back):
        predict( stock_prices_0[-days_back+i:], 0, 0, info_0[-days_back+i:])
    
#     start(1, "c2.h5")
#     print(predict( stock_prices_1, 0, 0, info_1))
    
#     start(2, "c3.h5")
#     print(predict( stock_prices_2, 0, 0, info_2))
    

In [89]:
main()

Starting model of company 0
Loading model from "models/c1.h5"
(0.0, 2019.0, 179.0, 1.0, 0.0, 1.0, 5.0, -1.2, 5765.0, 0.1, 0.0)
(0.0, 2019.0, 179.0, 1.0, 166.5)
(0.0, 2019.0, 179.0, 1.0, 166.5)
[[   0.     1.     5.    -1.2 5765.     0.1    0.     1.     0.     0. ]]
[0.         1.         0.28571429 0.44       0.57706597 0.55
 0.         0.         0.        ]
0
